In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
import os

from cloudpoints import load_points
from datasets import CloudDataset, label_names, df_results
from pathlib import Path

In [2]:
data_path = Path("3d_data/")

paris, paris_labels = load_points(data_path / "MiniParis1.ply")
lille1, lille1_labels = load_points(data_path / "MiniLille1.ply")
lille2, lille2_labels = load_points(data_path / "MiniLille2.ply")
dijon = load_points(data_path / "MiniDijon9.ply")
paris_wh = paris[:, 1] <= 20
paris1, paris1_label = paris[paris_wh], paris_labels[paris_wh]
paris2, paris2_label = paris[~paris_wh], paris_labels[~paris_wh]

datasets = [
    CloudDataset("paris1", paris1, paris1_label),
    CloudDataset("paris2", paris2, paris2_label),
    CloudDataset("lille1", lille1, lille1_labels),
    CloudDataset("lille2", lille2, lille2_labels),
    CloudDataset("dijon", dijon)
]
datasets

[CloudDataset<paris1, 1896865 points, labeled>,
 CloudDataset<paris2, 2262453 points, labeled>,
 CloudDataset<lille1, 1901853 points, labeled>,
 CloudDataset<lille2, 2500428 points, labeled>,
 CloudDataset<dijon, 3079187 points, unlabeled>]

In [3]:
inputs = [
    "paris1",
    "paris2",
    "lille1",
    "lille2"
]

def run_GRM_executable(exec_name, input, args=""):
    GRM_path = "GRM_data"
    os.system(f"./{exec_name} {args} < {GRM_path}/{input}_GRM.in > {GRM_path}/{input}_GRM.out")

def run_GRM_executable_all(exec_name, args=""):
    with Pool(len(inputs)) as p:
        p.starmap(run_GRM_executable, [(exec_name, input, args) for input in inputs])
        
def get_clf_labels(input):
    with open(f"GRM_data/{input}_GRM.in") as f:
        lines = [l.strip().split(" ") for l in f.readlines()]
    
    n, _ = lines[0]
    n = int(n)
    P = np.array([[float(f) for f in l] for l in lines[1:n+1]])
    return np.argmax(P, axis=1) + 1

Run your GRM executable : 

In [8]:
%%time
run_GRM_executable_all("repeated_tree_BP.out", "5.0 0.2 10")

CPU times: user 90.3 ms, sys: 120 ms, total: 210 ms
Wall time: 1min 45s


In [10]:
i = 1

GRM_labels = np.array(pd.read_csv(f"GRM_data/{inputs[i]}_GRM.out", header=None)[0])
true_labels = np.array(pd.read_csv(f"GRM_data/{inputs[i]}_labels.txt", header=None)[0])
clf_labels = get_clf_labels(inputs[i])

dataset = datasets[i]
dataset.write_ply_with_labels(f"viz_data/{inputs[i]}_clf.ply", clf_labels.astype(np.int32))
dataset.write_ply_with_labels(f"viz_data/{inputs[i]}_GRM.ply", GRM_labels.astype(np.int32))
dataset.write_ply_with_labels(f"viz_data/{inputs[i]}_true.ply", true_labels.astype(np.int32))

In [11]:
df = df_results(true_labels, GRM_labels)
print("Score:", df["IoU"].mean())
df

Score: 0.5000576169046954


,precision,recall,IoU
1 - Ground,0.961119,0.976041,0.945934
2 - Building,0.657976,0.935295,0.818093
3 - Poles,0.782003,0.287136,0.266795
4 - Pedestrians,0.511811,0.036907,0.035908
5 - Cars,0.871625,0.041863,0.041824
6 - Vegetation,0.909547,0.949207,0.891791


In [12]:
df = df_results(true_labels, clf_labels)
print("Score:", df["IoU"].mean())
df

Score: 0.4826614824699731


,precision,recall,IoU
1 - Ground,0.944321,0.968637,0.928423
2 - Building,0.634608,0.873523,0.731533
3 - Poles,0.497966,0.253888,0.209742
4 - Pedestrians,0.284959,0.127472,0.104072
5 - Cars,0.508233,0.115983,0.111357
6 - Vegetation,0.858031,0.897464,0.810842
